In [5]:
import collections
import math
import torch
import os, tarfile, json
import time, datetime
from io import StringIO
import numpy as np
import sagemaker
from sagemaker.pytorch import estimator, PyTorchModel, PyTorchPredictor, PyTorch
from sagemaker.utils import name_from_base
import boto3
from types import SimpleNamespace

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = 'privisaa-bucket-virginia'#sagemaker_session.default_bucket() # can replace with your own S3 bucket 'privisaa-bucket-virginia' # 
prefix = 'detectron2'
runtime_client = boto3.client('runtime.sagemaker')

In [4]:
#!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com


## Build our container 

In [3]:
%%sh

# The name of our algorithm
algorithm_name=detectron2-train

chmod +x train
chmod +x serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

# some kind of security auth issue with pushing this to ecr, not authorized to perform ecr:InitiateLayerUpload
docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  60.43MB
Step 1/21 : FROM nvidia/cuda:10.1-cudnn7-devel
 ---> b4879c167fc1
Step 2/21 : ENV DEBIAN_FRONTEND noninteractive
 ---> Using cache
 ---> 85bc36c63811
Step 3/21 : RUN apt-get update && apt-get install -y 	ca-certificates python3-dev git wget nginx sudo  	cmake ninja-build protobuf-compiler libprotobuf-dev &&   rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 10186fe31f54
Step 4/21 : RUN ln -sv /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> 13efb6c2b4ad
Step 5/21 : ENV PATH="/opt/ml/.local/bin:${PATH}"
 ---> Using cache
 ---> 1be291b0fe90
Step 6/21 : RUN wget https://bootstrap.pypa.io/get-pip.py && 	python3 get-pip.py && 	rm get-pip.py
 ---> Using cache
 ---> d03d837384b9
Step 7/21 : RUN pip install tensorboard cython boto3 numpy==1.15.4 gevent pandas flask pathlib gunicorn tqdm  requests six ipdb h5py html2text nltk progressbar onnxruntime sagemaker sagemaker_inference pycocotools git+https://github.com/NVIDIA/

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## Prep our input data

In [6]:
prefix = "mask-rcnn/sagemaker" #prefix in your S3 bucket

s3train = f's3://{bucket}/nfl-data/helmet_imgs'

train = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type=None, s3_data_type='S3Prefix') 
# 'application/tfrecord'

data_channels = {'train': train}

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


## Setup our estimator

In [7]:
# set our hyperparameters
hyperparameters = {
                  'num_gpus':8,
                  'config_file':'/opt/ml/detectron2/configs/quick_schedules/helmet_mask_rcnn_R_50_FPN_training_acc_test.yaml'
}

# instantiate model
torch_model = PyTorch( role=role,
                      train_instance_count=1,
                      train_instance_type= 'ml.p3dn.24xlarge', # try local
                      entry_point='/home/ec2-user/SageMaker/code/object-tracking-project/detectron2_train/detectron2/tools/plain_train_net.py',
                      image_name='209419068016.dkr.ecr.us-east-1.amazonaws.com/detectron2-train',
                      framework_version='1.5.1',
                      hyperparameters=hyperparameters
                     )


Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


## Train our model!

In [ ]:
torch_model.fit(inputs=data_channels)


In [6]:
import boto3
kvs = boto3.client('kinesisvideo')

response = kvs.get_data_endpoint(
  StreamName='test-video-stream',
  #StreamARN='arn:aws:kinesisvideo:us-east-1:209419068016:stream/test-video-stream/1586209281595',
  APIName='GET_MEDIA'
)

# 'PUT_MEDIA'|'LIST_FRAGMENTS'|'GET_MEDIA_FOR_FRAGMENT_LIST'|'GET_HLS_STREAMING_SESSION_URL'|'GET_DASH_STREAMING_SESSION_URL'|'GET_CLIP'

response